<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download datasets
! wget 'https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_train'
! wget 'https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_test_normal'
! wget 'https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_test_abnormal'

--2023-07-11 07:50:38--  https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253021 (247K) [text/plain]
Saving to: ‘hdfs_train’

hdfs_train          100%[===================>] 247.09K  --.-KB/s    in 0.01s   

2023-07-11 07:50:38 (24.1 MB/s) - ‘hdfs_train’ saved [253021/253021]

--2023-07-11 07:50:39--  https://raw.githubusercontent.com/wuyifan18/DeepLog/master/data/hdfs_test_normal
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28730917 (27M) [text/plain]
Sav

In [2]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import argparse
import os

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# count rows of the hdfs_train

with open('/content/hdfs_train', "r") as file:
    totaln=0
    for line in file:
        totaln += 1

print('There are a total of {} lines'.format(totaln))

There are a total of 4855 lines


important point:
if you need to modify the elements of the line later on, using a list would be more suitable. If you want to ensure that the elements remain immutable, using a tuple would be appropriate.

In [4]:
window_size = 10
num_sessions = 0
inputs = []
outputs = []
with open('/content/hdfs_train', 'r') as f:
        for line in f:
            print(line)
            num_sessions += 1
            line = [int(n) - 1 for n in line.strip().split()]
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                outputs.append(line[i + window_size])
            break

print(line)
print(inputs)
print(outputs)
print(num_sessions)

5 5 5 22 11 9 11 9 11 9 26 26 26 23 23 23 21 21 21 

[4, 4, 4, 21, 10, 8, 10, 8, 10, 8, 25, 25, 25, 22, 22, 22, 20, 20, 20]
[[4, 4, 4, 21, 10, 8, 10, 8, 10, 8], [4, 4, 21, 10, 8, 10, 8, 10, 8, 25], [4, 21, 10, 8, 10, 8, 10, 8, 25, 25], [21, 10, 8, 10, 8, 10, 8, 25, 25, 25], [10, 8, 10, 8, 10, 8, 25, 25, 25, 22], [8, 10, 8, 10, 8, 25, 25, 25, 22, 22], [10, 8, 10, 8, 25, 25, 25, 22, 22, 22], [8, 10, 8, 25, 25, 25, 22, 22, 22, 20], [10, 8, 25, 25, 25, 22, 22, 22, 20, 20]]
[25, 25, 25, 22, 22, 22, 20, 20, 20]
1


In [5]:
# what happen if the line is smaller than windows size or equal

line = '5 5 5 22 11 9 11'
window_size = 10
num_sessions = 0
inputs = []
outputs = []

line = [int(n) - 1 for n in line.strip().split()]
for i in range(len(line) - window_size):
    inputs.append(line[i:i + window_size])
    outputs.append(line[i + window_size])

print(line)
print(inputs)
print(outputs)


[4, 4, 4, 21, 10, 8, 10]
[]
[]


In [6]:
line = '5 5 5 22 11 9 11 9 11 9'
window_size = 10
num_sessions = 0
inputs = []
outputs = []

line = [int(n) - 1 for n in line.strip().split()]
for i in range(len(line) - window_size):
    inputs.append(line[i:i + window_size])
    outputs.append(line[i + window_size])

print(line)
print(inputs)
print(outputs)

[4, 4, 4, 21, 10, 8, 10, 8, 10, 8]
[]
[]


In [7]:
import torch
from torch.utils.data import TensorDataset

name = 'hdfs_train'
window_size = 10
num_sessions = 0
inputs = []
outputs = []

with open('/content/' + name, 'r') as f:
        for row in f:
            num_sessions += 1
            line = [int(i) - 1 for i in row.strip().split()] # we substract by one from templates index for starting from zero
            for i in range(len(line) - window_size):
                inputs.append(line[i:i + window_size])
                outputs.append(line[i + window_size])

print('Number of sessions({}): {}'.format(name, num_sessions))
print('Number of seqs({}): {}'.format(name, len(inputs)))
dataset = TensorDataset(torch.tensor(inputs, dtype=torch.float), torch.tensor(outputs))


Number of sessions(hdfs_train): 4855
Number of seqs(hdfs_train): 46575


In [8]:
# ba carefull, we substract by one
inputs[0]

[4, 4, 4, 21, 10, 8, 10, 8, 10, 8]

In [9]:
outputs[0]

25

In [10]:
import numpy as np
print(np.max(outputs))
print(np.min(outputs))

25
1


In [11]:
# unique outputs
print(set(outputs))
len(set(outputs))

{1, 2, 3, 4, 5, 8, 10, 15, 17, 20, 21, 22, 24, 25}


14

In [12]:
dataset[0]

(tensor([ 4.,  4.,  4., 21., 10.,  8., 10.,  8., 10.,  8.]), tensor(25))

In [13]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [14]:
input_size = 1 # means each input has one feature(template's code)
num_layers = 2
hidden_size = 64
num_classes = 28
batch_size = 2048
num_epochs = 50
model_dir = 'model'

In [15]:
log = 'Adam_batch_size={}_epoch={}'.format(str(batch_size), str(num_epochs))
writer = SummaryWriter(log_dir='log/' + log)


model = Model(input_size, hidden_size, num_layers, num_classes).to(device)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [16]:
for step, (seq, label) in enumerate(dataloader):
  print(seq.shape)
  print(label.shape)
  break

torch.Size([2048, 10])
torch.Size([2048])


In [ ]:
# Train the model
start_time = time.time()
total_step = len(dataloader)
for epoch in range(num_epochs):  # Loop over the dataset multiple times
    train_loss = 0
    for step, (seq, label) in enumerate(dataloader):
        # Forward pass
        seq = seq.clone().detach().view(-1, window_size, input_size).to(device)
        print()
        output = model(seq)
        loss = criterion(output, label.to(device))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        writer.add_graph(model, seq)
    print('Epoch [{}/{}], train_loss: {:.4f}'.format(epoch + 1, num_epochs, train_loss / total_step))
    writer.add_scalar('train_loss', train_loss / total_step, epoch + 1)
elapsed_time = time.time() - start_time
print('elapsed_time: {:.3f}s'.format(elapsed_time))
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
torch.save(model.state_dict(), model_dir + '/' + log + '.pt')
writer.close()
print('Finished Training')

In [30]:
torch.save(model.state_dict(), '/content/model_parameter')

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())


Model's state_dict:
lstm.weight_ih_l0 	 torch.Size([256, 1])
lstm.weight_hh_l0 	 torch.Size([256, 64])
lstm.bias_ih_l0 	 torch.Size([256])
lstm.bias_hh_l0 	 torch.Size([256])
lstm.weight_ih_l1 	 torch.Size([256, 64])
lstm.weight_hh_l1 	 torch.Size([256, 64])
lstm.bias_ih_l1 	 torch.Size([256])
lstm.bias_hh_l1 	 torch.Size([256])
fc.weight 	 torch.Size([28, 64])
fc.bias 	 torch.Size([28])


In [45]:
x,y = dataset[0]
print(x)
print(y)

tensor([ 4.,  4.,  4., 21., 10.,  8., 10.,  8., 10.,  8.])
tensor(25)


In [48]:
model.eval()
with torch.no_grad():
  output = model(x.view(-1, 10, 1).to(device))

print(output)
print(np.argmax(output.cpu()))

tensor([[-3.0041,  0.3163,  1.0136,  1.7953, -2.0938, -1.6256, -3.8058, -3.6497,
          2.4321, -3.2977,  0.6449, -3.3084, -3.4796, -2.5271, -3.7358, -1.5612,
         -3.1032, -1.6390, -3.5196, -3.6346, -0.9240, -2.0603, -0.4215, -2.8204,
         -0.8396,  9.3655, -3.0329, -3.1513]], device='cuda:0')
tensor(25)


In [51]:
x,y = dataset[5]
print(x)
print(y)

tensor([ 8., 10.,  8., 10.,  8., 25., 25., 25., 22., 22.])
tensor(22)


In [52]:
model.eval()
with torch.no_grad():
  output = model(x.view(-1, 10, 1).to(device))

print(output)
print(np.argmax(output.cpu()))

tensor([[-3.4347,  1.9512,  1.4356,  1.3532,  2.1123, -0.9423, -4.2348, -3.3266,
         -1.3622, -3.6461, -0.8179, -2.8601, -3.2928, -3.0511, -3.1072, -1.3533,
         -3.0274,  1.8199, -3.3067, -3.1413,  2.7089, -2.4208,  8.5549, -3.1654,
          0.8736, -1.1496, -2.8451, -2.9724]], device='cuda:0')
tensor(22)
